In [ ]:
import random

from pydantic_ai import Agent, RunContext
from pydantic_ai.providers.openai import OpenAIProvider
from pydantic_ai.models.openai import OpenAIModel

# provider that talks to llama-cpp
llama = OpenAIProvider(
    base_url="http://llama_cpp:8000/v1",
    api_key="local",
)

model = OpenAIModel(
    "Nous-Hermes-2-DPO_into_Nous_Hermes-2-Pro.Q8_0.gguf", 
    provider=llama,
)

agent = Agent(
    model,
    deps_type=str,
    instructions=(
        "You're a dice game, you should roll the dice and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Use the player's name in the response."
    ),
)

In [ ]:
@agent.tool_plain  
def roll_dice() -> str:
    """Roll a six-sided dice and return the result."""
    return str(random.randint(1, 6))


@agent.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps

In [ ]:
dice_result = await agent.run('My guess is 4', deps='Anne')  

print(dice_result.output)
#> Congratulations Anne, you guessed correctly! You're a winner!

In [ ]:
for m in dice_result.all_messages():
    print(m, "\n")